In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torchvision import models

In [2]:
## 파라메타 설정

In [3]:
batch_size = 128
epochs = 50
lr = 0.001

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(444)
if device == 'cuda':
    torch.cuda.manual_seed_all(444)
print(device + ' is avaulable')

cuda is avaulable


In [5]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True, num_workers = 2)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False, num_workers = 2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
num_classes = 10

Files already downloaded and verified
Files already downloaded and verified


In [6]:
## ECOC multi-hot encoding

In [7]:
label_encoding = torch.FloatTensor([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                                    [0, 1, 1, 0, 1, 1, 0, 0, 0 ,0],
                                    [0, 0, 1, 1, 1, 0, 1, 0, 0, 0],
                                    [1, 0, 1, 0, 1, 0, 0 ,1 ,0 ,0],
                                    [0, 0, 0, 1, 0, 1, 0, 1, 1, 0],
                                    [0, 1, 1, 1, 0, 1, 0, 0, 0, 0],
                                    [0, 0, 0, 1, 1, 0, 1, 0, 1, 0],
                                    [0, 1, 0, 0, 1, 1, 0, 0, 0, 1],
                                    [0, 0, 0, 0, 1, 1, 1, 1, 0, 0],
                                    [1, 0, 0, 1, 0, 0, 0, 0, 1, 1]])

In [8]:
## Resnet18 모델 생성

In [9]:
resnet18 = models.resnet18(pretrained = True)
resnet18.fc = nn.Linear(512, num_classes)

C:\Users\fly4h\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\Users\fly4h\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
resnet18 = resnet18.cuda()
optimizer = optim.Adam(resnet18.parameters(), lr = lr, weight_decay = 0.00001)
criterion = nn.MSELoss()

In [11]:
## 모델 학습

In [12]:
label_encoding = label_encoding.cuda()

In [13]:
for epoch in range(epochs):
    train_loss = 0
    for i, (train_data, train_label) in enumerate(trainloader):
        
        train_data = train_data.cuda()
        train_label = train_label.cuda()
        train_ecoc_label = label_encoding[train_label]
        
        optimizer.zero_grad()
        
        train_output = resnet18(train_data)
        loss = criterion(train_output, train_ecoc_label)
        
        loss.backward()
        optimizer.step()
        
        train_loss = train_loss + loss.item()
        
    with torch.no_grad():
        correct = 0
        total = 0
        for i, (test_data, test_label) in enumerate(testloader):
            
            test_data = test_data.cuda()
            test_label = test_label.cuda()
            
            test_output = resnet18(test_data)
            
            total = total + len(test_label)
            for j in range(len(test_output)):
                correct = correct + ((test_output[j] * label_encoding).sum(1).max(0)[1] == test_label[j]).item()
            
        print("epoch : {:3d}, test acc : {:3.3f}%, train loss : {:3.4f}".format(epoch + 1, correct / total * 100, train_loss))

epoch :   1, test acc : 71.610%, train loss : 57.0692
epoch :   2, test acc : 75.800%, train loss : 34.5163
epoch :   3, test acc : 76.850%, train loss : 27.9506
epoch :   4, test acc : 77.670%, train loss : 24.3476
epoch :   5, test acc : 78.100%, train loss : 22.5976
epoch :   6, test acc : 78.670%, train loss : 18.5354
epoch :   7, test acc : 78.280%, train loss : 16.3953
epoch :   8, test acc : 78.550%, train loss : 17.4177
epoch :   9, test acc : 78.360%, train loss : 18.5896
epoch :  10, test acc : 80.150%, train loss : 13.8135
epoch :  11, test acc : 80.170%, train loss : 9.5866
epoch :  12, test acc : 79.870%, train loss : 8.5625
epoch :  13, test acc : 79.530%, train loss : 8.2100
epoch :  14, test acc : 79.590%, train loss : 8.7867
epoch :  15, test acc : 79.560%, train loss : 7.8812
epoch :  16, test acc : 79.840%, train loss : 6.7981
epoch :  17, test acc : 79.390%, train loss : 6.3856
epoch :  18, test acc : 80.070%, train loss : 6.4910
epoch :  19, test acc : 79.260%, tra

In [14]:
## 모델 저장

In [15]:
resnet18 = resnet18.cpu()
path = 'TrainedModel/Resnet18(ECOC).pt'
torch.save(resnet18, path)